In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.regression import LinearRegression
spark=SparkSession.builder.appName("Linear Regression").getOrCreate()

In [5]:
df=spark.read.csv("C:/Users/User/Desktop/SparkFolder/Data/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [7]:
df.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [9]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [13]:
df.describe().select(['Email','Address','Avatar','Avg Session Length','Time on App','Time on Website']).show()

+-----------------+--------------------+-----------+------------------+------------------+------------------+
|            Email|             Address|     Avatar|Avg Session Length|       Time on App|   Time on Website|
+-----------------+--------------------+-----------+------------------+------------------+------------------+
|              500|                 500|        500|               500|               500|               500|
|             null|                null|       null| 33.05319351819619|12.052487937166134| 37.06044542094859|
|             null|                null|       null|0.9925631110845354|0.9942156084725424|1.0104889067564033|
|aaron04@yahoo.com|0001 Mack MillNor...|  AliceBlue|29.532428967057943| 8.508152176032603| 33.91384724758464|
|zscott@wright.com|Unit 7502 Box 834...|YellowGreen| 36.13966248879052|15.126994288792467|40.005181638101895|
+-----------------+--------------------+-----------+------------------+------------------+------------------+



In [15]:
df.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [18]:
df.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [25]:
for i in df.head(2)[1]:
    print(i)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [26]:
# So in Spark our festures must sampled as Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [27]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [28]:
# For this exercise we are only interested on the numeric columns
df1=df.select (['Avg Session Length',\
 'Time on App',\
 'Time on Website',\
 'Length of Membership',\
 'Yearly Amount Spent'] )

In [29]:
df1.show(5)

+------------------+------------------+------------------+--------------------+-------------------+
|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+------------------+------------------+------------------+--------------------+-------------------+
| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
| 34.30555662975554|13.717513665142507| 36.72128267790313|   3.120178782748092|  581.8523440352177|
| 33.33067252364639|12.795188551078114| 37.53665330059473|   4.446308318351434|  599.4060920457634|
+------------------+------------------+------------------+--------------------+-------------------+
only showing top 5 rows



# Use Assembler to generate input and OutputCols
* Basically you are trying to generate a data structire like the libsvm

In [33]:
assembler=VectorAssembler(inputCols=['Avg Session Length',\
 'Time on App',\
 'Time on Website',\
 'Length of Membership'], outputCol='features')

In [36]:
#Transform Data
output=assembler.transform(df)

Exception ignored in: <function JavaWrapper.__del__ at 0x00000231CDD31790>
Traceback (most recent call last):
  File "C:\spark\python\pyspark\ml\wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


In [38]:
output.printSchema()  # A new column is created called features

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [39]:
output.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [41]:
output.select ( ["Email","features"]).show(5)  # You realise all the features have been vectorized in a form of a list


+--------------------+--------------------+
|               Email|            features|
+--------------------+--------------------+
|mstephenson@ferna...|[34.4972677251122...|
|   hduke@hotmail.com|[31.9262720263601...|
|    pallen@yahoo.com|[33.0009147556426...|
|riverarebecca@gma...|[34.3055566297555...|
|mstephens@davidso...|[33.3306725236463...|
+--------------------+--------------------+
only showing top 5 rows



In [45]:
output.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [47]:
output.head(1)[0][-1]  # So you see all the important features have been combined in an array or dense vectors.\
#That the standard format Spark wants

DenseVector([34.4973, 12.6557, 39.5777, 4.0826])